In [ ]:
# Задание № 1: pulsar_stars

# Выборка данных и нормировка

In [24]:
import pandas as pd
import numpy as np

import io
df2 = pd.read_csv('report.csv')

Укажите число строк в полученной выборке: 

In [25]:
new_ds = df2[((df2['TARGET'] == 0) & (df2['MIP'] >= 83) & (df2['MIP'] <= 84)) | ((df2['TARGET'] == 1) & (df2['MIP'] >= 83) & (df2['MIP'] <= 89))]
len(new_ds.index)

136

Определите выборочное среднее для столбца MIP после нормировки:

In [26]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

normal_ds = pd.DataFrame(scaler.fit_transform(new_ds))
round(np.mean(normal_ds[0]),3)

0.266

In [27]:
normal_ds.head()

,0,1,2,3,4,5,6,7,8
0,0.300680,0.267009,0.429193,0.306216,0.033361,0.247052,0.281124,0.061703,1.0
1,0.115646,0.148421,0.069739,0.213864,0.022596,0.152646,0.323719,0.089913,0.0
2,0.165986,0.030998,0.250464,0.397038,0.007346,0.033184,0.537325,0.326763,0.0
3,0.089796,0.349197,0.073905,0.098465,0.018706,0.095516,0.325534,0.103149,0.0
4,0.944218,0.195245,0.122403,0.163214,0.012278,0.097978,0.399278,0.143982,1.0


# Логическая регрессия

In [28]:
#отбор данных для предикторов, удаление дввух последних столбцов, индекс не включается в данные автоматически.
X = pd.DataFrame(normal_ds.drop([8], axis=1))

In [29]:
#отбор столбца для отклика
y = pd.DataFrame(normal_ds[8])

In [30]:
#подключение модели логистической регрессии из библиотеки sklearn
from sklearn.linear_model import LogisticRegression

In [31]:
#обучение модели
reg = LogisticRegression(random_state=2019, solver='lbfgs').fit(X, y.values.ravel())

Введите вероятность отнесения звезды к классу "не пульсар":


In [32]:
#предсказание для сладости, введенной вручную, вероятности указаны для классов 0 (не пульсар) и 1 (пульсар) соответственно 
reg.predict_proba([[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]])

array([[0.72644503, 0.27355497]])

# Расстояние от новой звезды до ближайшего соседа

In [33]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=136, p=1)
neigh.fit(X, y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


KNeighborsClassifier(n_neighbors=136, p=1)

In [34]:
NewObject = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]

In [35]:
round(neigh.kneighbors([NewObject])[0][0][0],3)

0.548

# Задание 2: рукописные буквы

In [1]:
!pip install --upgrade pip
!pip install --upgrade scikit-learn==0.23.0

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 7.3 MB 2.6 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


1. Для начала параметр n_components приравниваем к рандомному числу

In [19]:
pivot = 500

In [40]:

(X_train, y_train), (X_pred, y_pred) = mnist.load_data()

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=126)

dim = 784 # 28*28 
X_train = X_train.reshape(len(X_train), dim)
X_test = X_test.reshape(len(X_test), dim)

from sklearn.decomposition import PCA
pca = PCA(n_components = pivot, svd_solver='full')
modelPCA = pca.fit(X_train) 
X_train = modelPCA.transform(X_train)

2. Находим долю объясненной дисперсии в зависимости от количества используемых ГК.
Ищем в массиве нужное значение дисперсии. Например, по заданию надо, чтобы доля объясненной дисперсии превышала 0.83.
В массиве под индексом 52 находится такое число - 0.831. 
Значит, возвращаемся в предыдущий пункт и меняет значение параметра n_components на 52.
Индекс найденного числа в массиве есть ОТВЕТ НА ПЕРВЫЙ ПУНКТ.

In [22]:
dispersion = 0.83
explained_variance = np.round(np.cumsum(pca.explained_variance_ratio_),3)

#print(explained_variance)
it = 1
for i in explained_variance:
  if (i >= dispersion):
    pivot = it
    break
  it += 1

pivot

52

Затем фигачим многоклассовую классификацию one vs all


In [41]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

tree = RandomForestClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=126)
clf = OneVsRestClassifier(tree).fit(X_train, y_train)

X_test = modelPCA.transform(X_test)
y_pred = clf.predict(X_test)

from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_test, y_pred)

ОТВЕТ НА ПУНКТ 4: количество верно классифицированных объектов класса 5 для тестовых данных:

In [42]:
CM[5][5]

1414

In [43]:
#жалкая попытка... Введите выборочное среднее нулевой колонки для полученного тренировочного набора:

np.mean(X_test[0])

14.094429459288447

пункты 5 и 6

In [ ]:
new = pd.read_csv('pred_for_task.csv')

In [ ]:
new.head()

In [ ]:
X = pd.DataFrame(new.drop(['FileName','Label'], axis=1))
y = pd.DataFrame(new['Label'])

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=126)

In [ ]:
y_train.head()

In [ ]:
y_test.head()

In [ ]:
dim = 784 # 28*28 
X_train = X_train.values.reshape(len(X_train), dim)
X_test = X_test.values.reshape(len(X_test), dim)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=21, svd_solver='full')
modelPCA = pca.fit(X_train) 
X_train = modelPCA.transform(X_train)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

tree = RandomForestClassifier(criterion='gini', min_samples_leaf=10, max_depth=20, n_estimators=10, random_state=126)
clf = OneVsRestClassifier(tree).fit(X_train, y_train)

X_test = modelPCA.transform(X_test)
y_pred = clf.predict(X_test)

from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_test, y_pred)

In [ ]:

CM